In [ ]:
!pip install transformers
# !unzip data_sets.zip

In [ ]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
import random
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig, AutoTokenizer, TFBertModel
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
import heapq
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
# BERT is pretrained with a max length of 512, you can define a lower max length ~ 300


#I added 128 below
MAX_LEN = 256
PRETRAINED_MODEL_ID = "bert-base-uncased"
tf.random.set_seed(
    42
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def loadRawData(path):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    #Read in the negative reviews for training
    for file in os.listdir(path+'/training_set/neg/'):
        if file.endswith('.txt'):
            with open(path+'/training_set/neg/'+file) as f:
                raw = f.read()
                x_train.append(raw)
                y_train.append(0)

    #Read in the positive reviews for training
    for file in os.listdir(path + '/training_set/pos/'):
        if file.endswith('.txt'):
            with open(path + '/training_set/pos/' + file) as f:
                raw = f.read()
                x_train.append(raw)
                y_train.append(1)

    #Read in the negative reviews for testing
    for file in os.listdir(path + '/test_set/neg/'):
        if file.endswith('.txt'):
            with open(path + '/test_set/neg/' + file) as f:
                raw = f.read()
                x_test.append(raw)
                y_test.append(0)

    #Read in the positive reviews for training
    for file in os.listdir(path + '/test_set/pos/'):
        if file.endswith('.txt'):
            with open(path + '/test_set/pos/' + file) as f:
                raw = f.read()
                x_test.append(raw)
                y_test.append(1)

    # x_train = np.asarray(x_train)
    # y_train = np.asarray(y_train)
    # x_test = np.asarray(x_test)
    # y_test = np.asarray(y_test)

    shuffle = np.random.permutation(len(x_train))
    x_train1 =[]
    y_train1= []
    for i in shuffle:
        x_train1.append(x_train[i])
        y_train1.append(y_train[i])
    
    x_train= x_train1
    y_train = y_train1

    return x_train, x_test, y_train, y_test


The following function represents the reviews in a format accepted by BERT

In [ ]:
### Already implemented
def create_inputs_targets(text_examples, tokenizer):
    '''converts inputs into the representation accepted by BERT
    '''
    dataset_dict = {
        "input_ids": [],
        
        "attention_mask": []
    }
    
    for item in text_examples:
        encodings = tokenizer(item, return_tensors='tf', max_length = MAX_LEN, truncation = True, pad_to_max_length=True)
        
        dataset_dict["input_ids"].append((encodings.input_ids))
        dataset_dict["attention_mask"].append((encodings.attention_mask))
    
    for key in dataset_dict:
        dataset_dict[key] = np.squeeze(np.array(dataset_dict[key]), axis=1)
           
    x = [
        dataset_dict["input_ids"],
     
        dataset_dict["attention_mask"]
    ]
    
    return x


 We will use TFBertForSequenceClassification. This interface  enables us to load a pretrained BERT model with an added untrained single linear layer on top for classification that we will use as a classifier. We will finetune the entire pre-trained BERT model and the additional untrained classification layer for our Movie Review classification task. 

In [ ]:
## Already Implemented
def create_bert(args):
    '''Creates a model specified by pretrained_model_id

    '''
    # load pretrained BERT model 
    
    
    model = TFBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_ID, num_labels = 2 )
    
    
    # BERT inputs 
    input_ids = layers.Input(shape=(MAX_LEN,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(MAX_LEN,), dtype=tf.int32)
    
    embedding = model(
            input_ids,  attention_mask=attention_mask
        )[0]
 
    model = keras.Model(
        inputs=[input_ids, attention_mask],
        outputs=[embedding],
    )
    # from_logits=True computes the "softmax" activation as part of the cross-entropy loss layer
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = keras.optimizers.Adam(lr=args['learning_rate'])
    metrics = keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])
    return model




    

We will also compare to an MLP on BOW representation learnt in NaiveBayes Section. You can reuse code from HW3 for this portion.

In [ ]:
def create_mlp(args=None):
	# You can use args to pass parameter values to this method

	# Define model architecture
    input_shape = (100,)
    model = keras.Sequential()
    model.add(keras.layers.Dense(256, activation = 'sigmoid', input_shape = input_shape))
    model.add(keras.layers.Dense(128, activation= "sigmoid"))
    model.add(keras.layers.Dense(32, activation= "sigmoid"))
    model.add(keras.layers.Dense(1, activation= "sigmoid"))
    # model = Sequential()
	# model.add(Dense(units=, activation=, input_dim=))
	# add more layers...

	# Optimizer

    optimizer = keras.optimizers.Adam(lr=args['learning_rate'])
        
    # Compile
    model.compile(loss='binary_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])


    return model
        


def plot_history(history, plot_title=''):
    train_loss_history = history.history['loss']
    val_loss_history = history.history['val_loss']

    train_acc_history = history.history['accuracy']
    val_acc_history = history.history['val_accuracy']
    # plot
    plt.plot(train_loss_history)
    plt.plot(val_loss_history)
    plt.title('training loss and validation loss vs epoch number')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='best')
    plt.show()


#changed validation_split to 0.1 from 0.0
def train_model(x_train, y_train, model_type, validation_split=0.0, args=None):
    # You can use args to pass parameter values to this method
    if model_type == 'BERT':
        model = create_bert(args)
        history = model.fit((x_train[0],x_train[1]), y_train, epochs = 4, batch_size = 9, shuffle = True)
    else:
        model = create_mlp(args)
        history = model.fit(x_train, y_train, epochs = 20, batch_size = 5, shuffle = True)
    return model, history


In [ ]:
def train_and_select_model(x_train, y_train, model_type, grading_mode):
    """Optional method. You can write code here to perform a 
    parameter search, cross-validation, etc. """

    # you can define hyperparameters differently for BERT and MLP using if else. Note that for BERT, the architecture is fixed and no changes are required. 
    # you are free to change the learning rate, batch size, etc. for BERT. 

    args = {
        'batch_size': 8,
        'validation_split': 0.1,
		'epoch':4,
        'learning_rate': 0.00001
        }
    
    best_valid_acc = 0
    best_hyper_set = {}
    

    # x_train1 = [x_train[i] for i in shuffle]
    # y_train1 = [y_train[i] for i in shuffle] 

    ## Select best values for hyperparamters such as learning_rate, optimizer, hidden_layer, hidden_dim, regularization...
    if not grading_mode:
        if model_type == 'BERT':
            best_model, best_history = train_model(x_train, y_train, model_type, validation_split=args['validation_split'], args=args)
        else:
            args = {
            'batch_size': 8,
            'validation_split': 0.1,
		    'epoch':4,
            'learning_rate': 0.001
            }
            best_model, best_history = train_model(x_train, y_train, model_type, validation_split=args['validation_split'], args=args)                    
        # for learning_rate in lr_values:
        #     for opt in ['adam']:
        #         for other_hyper in other_hyper_set:  ## search over other hyperparameters
        #             args['opt'] = opt
        #             args['learning_rate'] = learning_rate
        #             args['other_hyper'] = other_hyper				
        #             model, history = train_model(x_train, y_train, model_type, validation_split=args['validation_split'], args=args)
        #             validation_accuracy = history.history['val_accuracy']									
        #             max_valid_acc = max(validation_accuracy)
        
        #             if max_valid_acc > best_valid_acc:
        #                 best_model = model
        #                 best_valid_acc = max_valid_acc
        #                 best_hyper_set['learning_rate'] = learning_rate
        #                 best_hyper_set['opt'] = opt
        #                 best_history = history
    else:
        if model_type == 'BERT':
            best_model, best_history = train_model(x_train, y_train, model_type, validation_split=args['validation_split'], args=args)
        else:
            args = {
            'batch_size': 8,
            'validation_split': 0.1,
		    'epoch':4,
            'learning_rate': 0.001
            }
            best_model, best_history = train_model(x_train, y_train, model_type, validation_split=args['validation_split'], args=args)
        
            
        
    return best_model, best_history

Now, we will train both models and compare performance. 

In [ ]:
def processData(fullDatasetPath, model_type):
  
  # we will use the bert-base-uncased tokenizer - 
  if model_type == 'BERT':
    # load datasets as lists of reviews
    # no preprocessing required!
    # we will use the bert-base-uncased tokenizer - 
    XtrainText, XtestText, ytrain, ytest = loadRawData(fullDatasetPath)
    tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_ID)
    
    # tokenize for bert
    xtrain = create_inputs_targets(XtrainText, tokenizer)
    ytrain = np.array(ytrain)

    xtest = create_inputs_targets(XtestText, tokenizer)
    ytest = np.array(ytest)
    
  else:
    # reuse data loading functions from NaiveBayes portions, you can optionally save the BOW matrices and reload here 
    XtrainText, XtestText, ytrain, ytest, vocabulary = loadData(fullDatasetPath)
    MIN_FREQ = 3
    MAX_VOCAB = 100
    vocabulary = dict((word, index) for word, index in vocabulary.items() if
                      vocabulary[word]>= MIN_FREQ and word in heapq.nlargest(MAX_VOCAB,
                                                                                      vocabulary, key=vocabulary.get))
    xtrain, xtest = getBOWRepresentation(XtrainText, XtestText, vocabulary) 
    
  return xtrain, xtest, ytrain, ytest



# Finetune pretrained BERT and train MLP from scratch.

In [ ]:
def loadData(path):
    '''
    reads data from the folders
    x_train : [review1, review2, ....., review_n], where each review1 is a list of tokens

    vocabulary is a dictionary: (key: word, value: count)
    '''
    x_train = []
    y_train = []
    vocabulary = []
    x_test = []
    y_test = []
    #Read in the negative reviews for training
    for file in os.listdir(path+'/training_set/neg/'):
      if file.endswith('.txt'):
          with open(path+'/training_set/neg/'+file) as fil:
              raw = fil.read().strip()
              unstemmed_tokens = tokenizer.tokenize(raw)
              tokens1 = [ps.stem(word) for word in unstemmed_tokens]
              tokens = [word for word in tokens1 if word not in stop_words]
              vocabulary += tokens
              x_train += [tokens]
              y_train.append(0)

    #Read in the positive reviews for training
    for file in os.listdir(path+'/training_set/pos/'):
      if file.endswith('.txt'):
          with open(path+'/training_set/pos/'+file) as fil:
              raw = fil.read().strip()
              unstemmed_tokens = tokenizer.tokenize(raw)
              tokens1 = [ps.stem(word) for word in unstemmed_tokens]
              tokens = [word for word in tokens1 if word not in stop_words]
              vocabulary += tokens
              x_train += [tokens]
              y_train.append(1)

    #Read in the negative reviews for testing
    for file in os.listdir(path+'/test_set/neg/'):
      if file.endswith('.txt'):
          with open(path+'/test_set/neg/'+file) as fil:
              raw = fil.read().strip()
              unstemmed_tokens = tokenizer.tokenize(raw)
              tokens1 = [ps.stem(word) for word in unstemmed_tokens]
              tokens = [word for word in tokens1 if word not in stop_words]
              vocabulary += tokens
              x_test += [tokens]
              y_test.append(0)

    #Read in the positive reviews for training
    for file in os.listdir(path+'/test_set/pos/'):
      if file.endswith('.txt'):
          with open(path+'/test_set/pos/'+file) as fil:
              raw = fil.read().strip()
              unstemmed_tokens = tokenizer.tokenize(raw)
              tokens1 = [ps.stem(word) for word in unstemmed_tokens]
              tokens = [word for word in tokens1 if word not in stop_words]
              vocabulary += tokens
              x_test += [tokens]
              y_test.append(1)

    # Convert vocabulary list to dictionary
    vocab_dict = {}
    for word in vocabulary:
      if word in vocab_dict:
          vocab_dict[word] = vocab_dict[word]+1
      else:
          vocab_dict[word] = 1
    shuffle = np.random.permutation(len(x_train))
    x_train = [x_train[i] for i in shuffle]
    y_train = [y_train[i] for i in shuffle]
    return x_train, x_test, y_train, y_test, vocab_dict

def getBOWRepresentation(x_train, x_test, vocabulary):
    '''
    converts into Bag of Words representation
    each column is a feature(unique word) from the vocabulary
    x_train_bow : a numpy array with bag of words representation
    '''
    x_train_bow = np.zeros([len(x_train), len(vocabulary)])
    for i,review in enumerate(x_train):
        for j,word in enumerate(vocabulary):
            x_train_bow[i][j] = review.count(word)

    x_test_bow = np.zeros([len(x_test), len(vocabulary)])
    for i,review in enumerate(x_test):
        for j,word in enumerate(vocabulary):
            x_test_bow[i][j] = review.count(word)

    return np.array(x_train_bow), np.array(x_test_bow)


In [ ]:
grading_mode = True
path = '/content/drive/My Drive/data_sets/'

if grading_mode:
    # BERT model 
    xtrain, xtest, ytrain, ytest = processData(path, 'BERT')
    model, history = train_and_select_model(xtrain, ytrain, 'BERT', grading_mode=True)
    loss_test, acc_test = model.evaluate(xtest, ytest)
    print(acc_test)
    # MLP model
    xtrain, xtest, ytrain, ytest = processData(path, 'MLP')
    vec_length = len(xtrain[0])
    xtrain = np.array(xtrain)
    ytrain = np.array(ytrain)
    xtest = np.array(xtest)
    ytest = np.array(ytest)
    model, history = train_and_select_model(xtrain, ytrain, 'MLP', grading_mode=True)
    loss_test, acc_test = model.evaluate(xtest, ytest)
    print(acc_test)

else:
  #BERT model 
    xtrain, xtest, ytrain, ytest = processData(path, 'BERT')
    model, history = train_and_select_model(xtrain, ytrain, 'BERT', grading_mode=False)
    loss_test, acc_test = model.evaluate(xtest, ytest)
    plot_history(history)

    #MLP model
    xtrain, xtest, ytrain, ytest = processData(path, 'MLP')
    vec_length = len(xtrain[0])
    xtrain = np.array(xtrain)
    ytrain = np.array(ytrain)
    xtest = np.array(xtest)
    ytest = np.array(ytest)
    model, history = train_and_select_model(xtrain, ytrain, 'MLP', grading_mode=False)
    loss_test, acc_test = model.evaluate(xtest, ytest)
    plot_history(history)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassifi

Epoch 1/4
156/156 [==============================] - 86s 554ms/step - loss: 0.6915 - accuracy: 0.5250
Epoch 2/4
156/156 [==============================] - 86s 552ms/step - loss: 0.5452 - accuracy: 0.7307
Epoch 3/4
156/156 [==============================] - 86s 554ms/step - loss: 0.3289 - accuracy: 0.8643
Epoch 4/4
19/19 [==============================] - 11s 576ms/step - loss: 0.4739 - accuracy: 0.8233
0.8233333230018616
Epoch 1/20
280/280 [==============================] - 1s 2ms/step - loss: 0.6904 - accuracy: 0.5186
Epoch 2/20
280/280 [==============================] - 1s 2ms/step - loss: 0.6787 - accuracy: 0.6143
Epoch 3/20
280/280 [==============================] - 1s 2ms/step - loss: 0.6635 - accuracy: 0.6686
Epoch 4/20
280/280 [==============================] - 1s 2ms/step - loss: 0.6396 - accuracy: 0.6929
Epoch 5/20
280/280 [==============================] - 1s 2ms/step - loss: 0.6126 - accuracy: 0.6943
Epoch 6/20
280/280 [==============================] - 1s 2ms/step - loss: 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')